In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import logging
import pygenie
from pygenie.conf import GenieConf
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(__name__)

config_file = "conf/pygenie_test.ini"
conf = GenieConf(config_file=config_file)

# 根据genie配置修改user name
job = pygenie.jobs.GenieJob(conf) \
    .job_name('Genie Demo Spark Job') \
    .genie_username('kongly') \
    .job_version('3.0.0')

job.cluster_tags(['sched:bdp-cce2-test-sg','type:k8s'])
job.command_tags(['type:spark-submit'])

# command = "--pod-config metastore.yaml --namespace bdp-test"
command = """--deploy-mode cluster \
        --name jupyterhub-test \
        --class TestFM_Push \
        --conf spark.executor.instances=100 \
        --conf spark.driver.memory=25G \
        --conf spark.executor.memory=47G \
        --conf spark.executor.cores=8 \
        --conf spark.dynamicAllocation.enabled=false \
        --conf spark.driver.maxResultSize=25G \
        --conf spark.kubernetes.allocation.batch.size=50 \
        obs://bdp-deploy-sg/kongly/MLFM.jar \
        """+train_path+" "+feature_path+" "+model_output_path+" "+train_predict_output+" "+feature_output

# 执行
job.command_arguments(command)

running_job = job.execute()
print('Job {} is {}'.format(running_job.job_id, running_job.status))
print(running_job.job_link)

# Block and wait until job is done
running_job.wait()
print('Job {} finished with status {}'.format(running_job.job_id, running_job.status))
for x in running_job.get_log('stdout', iterator=True):
    print(x)

NameError: name 'train_path' is not defined